# Model and Data import

In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("m-a-p/OpenCodeInterpreter-DS-6.7B")
model = AutoModelForCausalLM.from_pretrained("m-a-p/OpenCodeInterpreter-DS-6.7B")

c:\Users\mcandi\.conda\envs\models_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.66s/it]


In [1]:
import os

def extract_data(file_path):
    data = []
    with open(file_path, "r") as file:
        for line in file:
            data.append(line.strip())
    return data


def save_translations(translations, filename):
    file_path = os.path.join('results', f'{filename}.txt')
    # if not os.path.exists(file_path):
    #     os.makedirs(file_path)

    with open(file_path, 'w') as file:
        for item in translations:
            file.write(f"{item}\n")



python_data_path = r'benchmark\G-TransEval\Python.test' 
java_data_path = r'benchmark\G-TransEval\Java.test' 

python_data = extract_data(python_data_path)
java_data = extract_data(java_data_path)


In [57]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("m-a-p/OpenCodeInterpreter-DS-6.7B")

max_lll = 0
for el in java_data:
    lll = len(tokenizer(el, return_tensors="pt")["input_ids"][0])
    if lll > max_lll:
        max_lll = lll

max_lll

c:\Users\mcandi\.conda\envs\models_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


512

# Translation extraction

In [197]:
import re

def extract_functions(file):

    with open(file, 'r') as file:
        data = file.read()
    split_data = data.split('\n\n# END OF TRANSLATION\n\n')[:-1]

    all_list = []

    for translation in split_data:
        if '```python' in translation:
            psttern = r'```python\n(.*?)```'
            match = re.findall(psttern, translation, re.DOTALL)[0]
            all_list.append(match)
        else:
            translated_function = 'def ' + translation.split('def ')[1]
            cleaned_translation = translated_function.split('#')[0]
            all_list.append(cleaned_translation)

    return all_list


def prepare_for_unit(data):
    for translation in data:
        translation = translation.replace('\n', 'NEW_LINE')

# Translation evaluation metrics

In [194]:
from codebleu import calc_codebleu

def BLUE_score(reference, prediction):
    result = calc_codebleu([reference], [prediction], lang="python", weights=(1, 0, 0, 0), tokenizer=None)
    return result['codebleu']


def CodeBLEU_score(reference, prediction):
    result = calc_codebleu([reference], [prediction], lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
    return result['codebleu']




def evaluation_metrics(reference, prediction):
    blue = BLUE_score(reference, prediction)
    code_blue = CodeBLEU_score(reference, prediction)

    result_metrics = {
        'BLUE': blue,
        'CodeBLEU': code_blue
    }

    return result_metrics



def reindent_code(code):

    indet_count = 0
    final_code = ''
    for line in code.split(' NEW_LINE '):

        if 'INDENT' in line:
            indet_count += line.count('INDENT')
            line = line.replace('INDENT ', '')
            line = '    ' * indet_count + line

        elif 'DEDENT' in line:
            indet_count -= line.count('DEDENT')
            line = line.replace('DEDENT ', '')
            line = '    ' * indet_count + line

        else:
            line = '    ' * indet_count + line

        final_code += line + '\n'

    return final_code



def format_code(code):

    code = code.replace('( ', '(').replace(' )', ')').replace(' ,', ',').replace(' :', ':'). replace(' . ', '.').replace('[ ', '[').replace(' ]', ']').replace('{ ', '{').replace(' }', '}')
    pattern = r'[a-zA-Z] \('
    code = re.sub(pattern, lambda match: match.group(0).replace(' (', '('), code)
    pattern = r'[a-zA-Z] \['
    code = re.sub(pattern, lambda match: match.group(0).replace(' [', '['), code)

    formatted_code = reindent_code(code)

    return formatted_code


In [200]:
with open(r'benchmark\G-TransEval\Python.test', 'r') as file:
    references = file.read()

references = references.split('\n')

predictions = extract_functions('baseline_translation.txt')


In [201]:
results = []
for reference, prediction in zip(references, predictions):
    reference = format_code(reference)
    prediction = format_code(prediction)
    metrics = evaluation_metrics(reference, prediction)
    results.append(metrics)

In [204]:
def calculate_mean_scores(data):
    total_blue = 0
    total_codebleu = 0
    count = len(data)

    for entry in data:
        total_blue += entry['BLUE']
        total_codebleu += entry['CodeBLEU']

    mean_blue = total_blue / count
    mean_codebleu = total_codebleu / count

    return {'BLUE': mean_blue, 'CodeBLEU': mean_codebleu}

mean_scores = calculate_mean_scores(results)
mean_scores

{'BLUE': 0.6607780209625196, 'CodeBLEU': 0.7438829946730859}

In [195]:
reference = format_code(references[0])
prediction = data[0]

evaluation_metrics(reference, prediction)

{'BLUE': 0.7976501109363606, 'CodeBLEU': 0.9010534742615681}

In [196]:
reference

'def greatest_common_divisor(a, b):\n    if a == 0 or b == 0:\n        return a + b\n    if a == b:\n        return a\n    if a > b:\n        return greatest_common_divisor(a % b, b)\n    else:\n        return greatest_common_divisor(a, b % a)\n\n'

# Parameters manipulation and count

In [90]:
for param in model.parameters():
    param.requires_grad = False

# for i in range(0, 31):
#     for param in model.model.layers[i].parameters():
#         param.requires_grad = True

for param in model.model.layers[0].parameters():
    param.requires_grad = True

In [96]:
total_param = sum(p.numel() for p in model.parameters())
trinable_param = sum(p.numel() for p in model.parameters() if p.requires_grad)

percentages = trinable_param / total_param * 100

print(f"Percentage of trainable parameters: {percentages:.2f}%")

Percentage of trainable parameters: 3.00%


In [98]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    trainable_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        trainable_params += params
    print(table)
    print(f"Total Trainable Params: {trainable_params}")
    total_param = sum(p.numel() for p in model.parameters())
    percentage = trainable_params/total_param * 100

    return print(f"\nPercentage of trainable parameters: {percentage:.2f}%")
    
count_parameters(model)

+------------------------------------------------+------------+
|                    Modules                     | Parameters |
+------------------------------------------------+------------+
|     model.layers.0.self_attn.q_proj.weight     |  16777216  |
|     model.layers.0.self_attn.k_proj.weight     |  16777216  |
|     model.layers.0.self_attn.v_proj.weight     |  16777216  |
|     model.layers.0.self_attn.o_proj.weight     |  16777216  |
|      model.layers.0.mlp.gate_proj.weight       |  45088768  |
|       model.layers.0.mlp.up_proj.weight        |  45088768  |
|      model.layers.0.mlp.down_proj.weight       |  45088768  |
|     model.layers.0.input_layernorm.weight      |    4096    |
| model.layers.0.post_attention_layernorm.weight |    4096    |
+------------------------------------------------+------------+
Total Trainable Params: 202383360

Percentage of trainable parameters: 3.00%
